## Peri-Event Time Histogram (PETH) with Pynapple

A **PETH** aligns neural activity to behavioral events and averages across trials. [Pynapple](https://pynapple.org) makes this simple:

- `nap.compute_perievent()` - align spikes to events
- `.to_tsd()` - flatten for raster plots  
- `.count(bin_size)` - bin spikes for firing rate

In [ ]:
import pynapple as nap
import numpy as np
import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient
import remfile
import h5py

# Stream NWB file from DANDI
dandiset_id = "001636"
session_id = "V++v4529++PostMPTP++Depth20600um++20000127"

client = DandiAPIClient()
dandiset = client.get_dandiset(dandiset_id)
assets = list(dandiset.get_assets())
asset = next(a for a in assets if session_id in a.path)
print(f"Streaming: {asset.path}")

# Open remote file
url = asset.get_content_url(follow_redirects=1, strip_query=True)
file = remfile.File(url)
h5_file = h5py.File(file, "r")
io = NWBHDF5IO(file=h5_file, load_namespaces=True)
nwbfile = io.read()
data = nap.NWBFile(nwbfile)
print(f"Session: {nwbfile.session_id}")

In [ ]:
# Extract data
trials = data["trials"]
spikes = data["units"][0]
velocity = data["ElbowVelocity"]

# Split by movement type
flex_trials = trials[trials["movement_type"] == "flexion"]
ext_trials = trials[trials["movement_type"] == "extension"]
flex_onset = nap.Ts(np.sort(flex_trials["derived_movement_onset_time"].values))
ext_onset = nap.Ts(np.sort(ext_trials["derived_movement_onset_time"].values))

# Parameters
window = (-0.5, 1.0)

# Align spikes and velocity to movement onset
peth_flex = nap.compute_perievent(spikes.restrict(flex_trials), tref=flex_onset, minmax=window)
peth_ext = nap.compute_perievent(spikes.restrict(ext_trials), tref=ext_onset, minmax=window)
vel_flex = nap.compute_perievent_continuous(velocity.restrict(flex_trials), tref=flex_onset, minmax=window)
vel_ext = nap.compute_perievent_continuous(velocity.restrict(ext_trials), tref=ext_onset, minmax=window)

print(f"Flexion: {len(flex_trials)} trials, Extension: {len(ext_trials)} trials")

In [ ]:
# Flatten across trials for raster plot (values = trial index, times = spike times)
raster_flexion = peth_flex.to_tsd()
raster_extension = peth_ext.to_tsd()

# Bin and smooth firing rate, then average across trials
bin_size = 0.01
std = 0.025  # For Gaussian smoothing
firing_rate_flexion = np.nanmean(peth_flex.count(bin_size).smooth(std=std).values, axis=1) / bin_size
firing_rate_extension = np.nanmean(peth_ext.count(bin_size).smooth(std=std).values, axis=1) / bin_size
t_fr = peth_flex.count(bin_size).times() * 1000

# Plot PETH
BLUE, RED = "#2563EB", "#DC2626"

fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
window_ms = (window[0]*1000, window[1]*1000)

# Row 1: Velocity
t_ms = vel_flex.times() * 1000
axes[0].plot(t_ms, np.nanmean(vel_flex.values, 1), lw=3, color=BLUE, label=f"Flexion (n={len(flex_trials)})")
axes[0].plot(t_ms, np.nanmean(vel_ext.values, 1), lw=3, color=RED, label=f"Extension (n={len(ext_trials)})")
axes[0].axvline(0, color='k', ls='--', lw=2)
axes[0].axhline(0, color='gray', alpha=0.3)
axes[0].set_ylabel("Velocity (deg/s)", fontsize=14, fontweight='bold')
axes[0].set_title("Elbow Velocity", fontsize=16, fontweight='bold')
axes[0].legend(fontsize=14, frameon=False)

# Row 2: Raster
axes[1].plot(raster_flexion.times()*1000, raster_flexion.values, '|', ms=8, mew=1.5, color=BLUE)
axes[1].plot(raster_extension.times()*1000, raster_extension.values + len(peth_flex), '|', ms=8, mew=1.5, color=RED)
axes[1].axhline(len(peth_flex)-0.5, color='gray', lw=2)
axes[1].axvline(0, color='k', ls='--', lw=2)
axes[1].set_ylabel("Trial", fontsize=14, fontweight='bold')
axes[1].set_title("Spike Raster", fontsize=16, fontweight='bold')

# Row 3: Firing Rate
axes[2].plot(t_fr, firing_rate_flexion, lw=3, color=BLUE)
axes[2].plot(t_fr, firing_rate_extension, lw=3, color=RED)
axes[2].axvline(0, color='k', ls='--', lw=2)
axes[2].set_xlabel("Time from movement onset (ms)", fontsize=14, fontweight='bold')
axes[2].set_ylabel("Firing Rate (Hz)", fontsize=14, fontweight='bold')
axes[2].set_title("PETH", fontsize=16, fontweight='bold')
axes[2].set_ylim(bottom=0)

for ax in axes:
    ax.set_xlim(*window_ms)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(labelsize=12)

plt.tight_layout()
plt.show()

### How It Works

The PETH above was created with just a few lines of code thanks to pynapple's data model:

**Core Data Types:**
- **`Ts`** - Timestamps (e.g., spike times, event times)
- **`Tsd`** - Time series with values (e.g., velocity, LFP)
- **`TsGroup`** - Collection of Ts objects (e.g., multiple neurons, or trials after `compute_perievent`)
- **`IntervalSet`** - Time intervals (e.g., trials, epochs)

**Why so few lines?**

1. **`nap.NWBFile()`** automatically loads NWB data into the right pynapple types
2. **`.restrict(intervals)`** filters any time series to specific intervals (no manual indexing)
3. **`compute_perievent()`** handles all the alignment math internally
4. **`.to_tsd()`** and **`.count().smooth()`** transform TsGroups for plotting in one call

| Function | What it does |
|----------|--------------|
| `compute_perievent(spikes, tref, minmax)` | Align spikes to events, returns TsGroup (one Ts per trial) |
| `compute_perievent_continuous(signal, tref, minmax)` | Align continuous data, returns TsdFrame (timepoints x trials) |
| `.to_tsd()` | Flatten TsGroup for raster (times = spike times, values = trial index) |
| `.count(bin).smooth(std)` | Bin spikes and apply Gaussian smoothing |

**Note on `.to_tsd()` for raster plots:**

Pynapple doesn't include plotting functions - it's designed to work seamlessly with matplotlib. After `compute_perievent()`, you have a TsGroup where each element contains one trial's spike times. The `.to_tsd()` method flattens it into a single Tsd where:
- `.times()` = all spike times concatenated across trials
- `.values` = the trial index for each spike

This gives you exactly what `plt.plot(x, y, '|')` needs for a raster: `plt.plot(raster.times(), raster.values, '|')`

### EMG Aligned to Movement Onset

The same `compute_perievent_continuous` pattern works for any continuous signal. Here we align EMG to movement onset to see muscle activation timing relative to the neural activity above.

**Expected pattern:** EMG activation should precede or coincide with movement onset, with agonist muscles showing direction-selective responses.

In [ ]:
# Align EMG to movement onset (same pattern as velocity)
emg = data["EMGBrachioradialis"]
emg_flex = nap.compute_perievent_continuous(emg.restrict(flex_trials), tref=flex_onset, minmax=window)
emg_ext = nap.compute_perievent_continuous(emg.restrict(ext_trials), tref=ext_onset, minmax=window)

# Plot EMG
fig, ax = plt.subplots(figsize=(12, 4))
t_ms = emg_flex.times() * 1000
ax.plot(t_ms, np.nanmean(emg_flex.values, 1), lw=3, color=BLUE, label=f"Flexion (n={len(flex_trials)})")
ax.plot(t_ms, np.nanmean(emg_ext.values, 1), lw=3, color=RED, label=f"Extension (n={len(ext_trials)})")
ax.axvline(0, color='k', ls='--', lw=2)
ax.set_xlim(*window_ms)
ax.set_xlabel("Time from movement onset (ms)", fontsize=14, fontweight='bold')
ax.set_ylabel("EMG (a.u.)", fontsize=14, fontweight='bold')
ax.set_title("Brachioradialis EMG", fontsize=16, fontweight='bold')
ax.legend(fontsize=14, frameon=False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Align EMG to movement onset (same pattern as velocity)
emg = data["EMGTricepsLongus2"]
emg_flex = nap.compute_perievent_continuous(emg.restrict(flex_trials), tref=flex_onset, minmax=window)
emg_ext = nap.compute_perievent_continuous(emg.restrict(ext_trials), tref=ext_onset, minmax=window)

# Plot EMG
fig, ax = plt.subplots(figsize=(12, 4))
t_ms = emg_flex.times() * 1000
ax.plot(t_ms, np.nanmean(emg_flex.values, 1), lw=3, color=BLUE, label=f"Flexion (n={len(flex_trials)})")
ax.plot(t_ms, np.nanmean(emg_ext.values, 1), lw=3, color=RED, label=f"Extension (n={len(ext_trials)})")
ax.axvline(0, color='k', ls='--', lw=2)
ax.set_xlim(*window_ms)
ax.set_xlabel("Time from movement onset (ms)", fontsize=14, fontweight='bold')
ax.set_ylabel("EMG (a.u.)", fontsize=14, fontweight='bold')
ax.set_title("Triceps Longus EMG", fontsize=16, fontweight='bold')
ax.legend(fontsize=14, frameon=False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(labelsize=12)
plt.tight_layout()
plt.show()